In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
import scBiMapping
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
adata = ad.read('/data/work/test_data/Human_Cancer_cell_lines_RNA.h5ad')  
adata.var_names_make_unique()
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes=4000,subset=True,flavor='cell_ranger') #
 
np.random.seed(seed=1)
id = np.arange(adata.n_obs); np.random.shuffle(id); adata = adata[id,:]
adata_ref = adata[:10000,:]
adata_query = adata[10000:,:]
print(adata_ref); print(adata_query)

View of AnnData object with n_obs × n_vars = 10000 × 14781
    obs: 'cancer_type'
View of AnnData object with n_obs × n_vars = 13101 × 14781
    obs: 'cancer_type'


## Case2： knnMethod = 'NNDescent'

In [3]:
%%time
n_embedding = 30; # number of embeddings
K = 30; # this parameter will heavily influence the speed. Don't be too large
K_majority = 5; # parameter for marjority voting
normalization = True; knnMethod = 'NNDescent'; metric = 'euclidean'
reduction_method_on_cells_only = 'BiMapping'; # 'BiMapping','SnapATAC2','None','minHash'

CellType_Key_for_ref = 'cancer_type' # this setting is Necessary, which denotes the cell type key of the reference dataset
scBiMapping.scBiMapping_annotation(adata_ref, adata_query,n_embedding = n_embedding,normalization = normalization,K = K,K_majority = K_majority,knnMethod = knnMethod,reduction_method_on_cells_only = reduction_method_on_cells_only,CellType_Key_for_ref =CellType_Key_for_ref)
print(adata_query.obs['cell_type_predicted'].head()) # predicted cell types for query cells

n_embedding:  30
normalization:  True
K:  30
knnMethod:  NNDescent
for each cell, find K nearest genes in the co-embedded space......
for each cell, find K nearest genes in the co-embedded space......
knn_based_Sim_ref:
(10000, 14781)
knn_based_Sim_query: 
(13101, 14781)

Direct merge softmax-weighted coded reference and query dataset......
(23101, 14781)

 reduction on sparse cell-markerGenes softmax-weighted matrix, to get low-embedding of cells only, using BiMapping
v5...
(10000, 30)
(13101, 30)

K_majority = 5 (for majority voting)
find knn...
voting...
CAGCCGTCACAGAGCGGCAA.CL200140333_L01                 K-562
AAGGTTCAATGTTACCACAT.DP8400020473BL_L01_80           SW480
CTACCGCGATCTTCAATAAT.CL200140455_L02                  HeLa
TCCTCGAGTAAGGTTAGGAG.DP8400012307BR_L01_15           HepG2
CGTCGGTTAATTGTGGCATC.CL200140333_L02          MDA-MB-231_2
Name: cell_type_predicted, dtype: object
CPU times: user 10min 35s, sys: 1min 14s, total: 11min 49s
Wall time: 3min 13s


In [4]:
# Evaluate 
accuracy = accuracy_score(adata_query.obs['cancer_type'], adata_query.obs['cell_type_predicted'])
print(f'accuracy: {accuracy}')

accuracy: 0.9173345546141516
